In [13]:
import numpy as np
import yfinance as yf

np.float_ = np.float64
from prophet import Prophet

import pandas as pd


# Coleta os dividendos históricos de um ativo.
def get_dividend_history(ticker):
    # Coletar dados do ticker
    stock = yf.Ticker(ticker)
    dividends = stock.dividends

    # Verificar se há dados de dividendos
    if dividends.empty:
        return pd.DataFrame(columns=['date', 'dividend'])

    # Ajustar os dados
    dividends = dividends.reset_index()
    dividends.columns = ['date', 'dividend']
    dividends['date'] = pd.to_datetime(dividends['date']).dt.tz_localize(None)

    return dividends


# Função para coletar dados do índice e calcular a média anual
def get_average_annual_return(index_ticker='^BVSP', period='10y'):
    index_data = yf.Ticker(index_ticker).history(period=period)

    index_data['Year'] = index_data.index.year
    annual_mean = index_data.groupby('Year')['Close'].mean()
    average_annual_return = annual_mean.pct_change().mean()

    return average_annual_return


# Função para calcular a taxa de crescimento dos dividendos usando Prophet
def get_dividend_growth_rate(ticker):
    dividends = get_dividend_history(ticker)
    dividends.columns = ['ds', 'y']

    # Criar e ajustar o modelo Prophet
    model = Prophet(growth='linear', yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False)
    model.fit(dividends)

    # Prever para o futuro e obter a taxa de crescimento
    future = model.make_future_dataframe(periods=1, freq='YE')  # Adiciona um ano
    forecast = model.predict(future)

    annual_forecast = forecast.groupby(pd.Grouper(key='ds', freq='YE')).sum()['trend']
    growth_rate = annual_forecast[annual_forecast > 0].pct_change().median()

    new_dividends = forecast[['ds', 'yhat']]
    new_dividends.columns = ['date', 'dividend']

    return new_dividends, growth_rate


# Função para calcular o custo de equidade (COE)
def calculate_coe(ticker):
    market_return = get_average_annual_return()
    risk_free_rate = 0.03
    market_risk_premium = market_return - risk_free_rate

    ticker_info = yf.Ticker(ticker).info
    beta = ticker_info.get("beta", 1)  # Use 1 as default if beta is not available

    coe = round(beta * market_risk_premium + risk_free_rate, 4)
    return coe


# Função para calcular o valor justo da ação usando o Modelo de Gordon
def calculate_fair_value(ticker):
    coe = calculate_coe(ticker)
    dividends, dividend_growth_rate = get_dividend_growth_rate(ticker)

    print(f'Cost of Equity (COE) for {ticker}: {coe:.4f}')
    print(f'Dividend growth rate for {ticker}: {dividend_growth_rate:.4f}')

    current_date = dividends['date'].max()
    filter_last_12_months = (dividends['date'] >= current_date - pd.DateOffset(months=12))
    last_12_months_dividends = dividends[filter_last_12_months]

    total_dividends_last_12_months = last_12_months_dividends['dividend'].sum()
    print(f'Total dividends in the last 12 months for {ticker}: {total_dividends_last_12_months:.2f}')

    fair_value = total_dividends_last_12_months * (1 + dividend_growth_rate) / abs(coe - dividend_growth_rate)

    return fair_value

# Testing

In [11]:
get_dividend_history("EGIE3.SA")

,date,dividend
0,2007-12-14,0.107853
1,2009-08-14,0.426516
2,2009-11-18,0.237766
3,2010-05-07,0.100392
4,2010-10-01,0.350644
5,2010-11-18,0.269632
6,2011-05-02,0.193975
7,2011-08-11,0.806464
8,2011-11-18,0.311290
9,2012-04-16,0.317026


In [16]:
# Exemplo de uso
test = ["VALE3.SA", "EGIE3.SA", "B3SA3.SA", "BBAS3.SA", "BBSE3.SA", "TRPL4.SA"]

for ticker in test:
    price = calculate_fair_value(ticker)
    print(f'O valor justo da ação {ticker} é: {price:.2f}')

17:34:02 - cmdstanpy - INFO - Chain [1] start processing
17:34:02 - cmdstanpy - INFO - Chain [1] done processing


Cost of Equity (COE) for VALE3.SA: 0.0892
Dividend growth rate for VALE3.SA: 0.1243
Total dividends in the last 12 months for VALE3.SA: 5.58
O valor justo da ação VALE3.SA é: 178.69


17:34:03 - cmdstanpy - INFO - Chain [1] start processing
17:34:03 - cmdstanpy - INFO - Chain [1] done processing


Cost of Equity (COE) for EGIE3.SA: 0.0638
Dividend growth rate for EGIE3.SA: 0.0903
Total dividends in the last 12 months for EGIE3.SA: 1.77
O valor justo da ação EGIE3.SA é: 72.73


17:34:04 - cmdstanpy - INFO - Chain [1] start processing
17:34:04 - cmdstanpy - INFO - Chain [1] done processing


...

Cost of Equity (COE) for BBAS3.SA: 0.0816
Dividend growth rate for BBAS3.SA: 0.0618
Total dividends in the last 12 months for BBAS3.SA: 0.91
O valor justo da ação BBAS3.SA é: 48.76


17:34:06 - cmdstanpy - INFO - Chain [1] start processing
17:34:06 - cmdstanpy - INFO - Chain [1] done processing


Cost of Equity (COE) for BBSE3.SA: 0.0721
Dividend growth rate for BBSE3.SA: 0.0912
Total dividends in the last 12 months for BBSE3.SA: 2.74
O valor justo da ação BBSE3.SA é: 156.69


17:34:07 - cmdstanpy - INFO - Chain [1] start processing
17:34:07 - cmdstanpy - INFO - Chain [1] done processing


Cost of Equity (COE) for TRPL4.SA: 0.0578
Dividend growth rate for TRPL4.SA: 0.1455
Total dividends in the last 12 months for TRPL4.SA: 1.74
O valor justo da ação TRPL4.SA é: 22.70
